In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import F1Score, BinaryF1Score, BinaryConfusionMatrix
import torch.optim.lr_scheduler as lr_scheduler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # 라벨 인코더
from sklearn.preprocessing import OneHotEncoder


TrainDF = pd.read_csv('TrainDF.csv')
TestDF = pd.read_csv('TestDF.csv')
TrainDF.dropna()
TestDF.dropna()

# 짜투리 전처리 location drop

# 데이터 날짜 -> 시간으로 변경
TrainDF['Date'][TrainDF['Date'].str[-2:]=="AM"] = TrainDF['Date'].str[-11:-9]
PMDF = TrainDF['Date'][TrainDF['Date'].str[-2:]=="PM"].str[-11:-9]
PMDF = PMDF.astype("int64")+12
TrainDF['Date'][TrainDF['Date'].str[-2:]=="PM"] = PMDF


0         11/25/2005 06:30:00 AM
1         12/03/2017 07:05:00 AM
2         11/12/2010 04:00:00 PM
3         09/16/2008 11:00:00 AM
4         04/29/2005 11:30:00 PM
                   ...          
124995    03/27/2006 06:45:00 AM
124996    01/07/2009 03:00:00 PM
124997    07/18/2012 06:00:00 AM
124998    03/22/2017 06:00:00 PM
124999    10/05/2002 09:25:52 AM
Name: Date, Length: 125000, dtype: object


C:\Users\kjy19\AppData\Local\Temp\ipykernel_47584\3254723030.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TrainDF['Date'][TrainDF['Date'].str[-2:]=="AM"] = TrainDF['Date'].str[-11:-9]
C:\Users\kjy19\AppData\Local\Temp\ipykernel_47584\3254723030.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TrainDF['Date'][TrainDF['Date'].str[-2:]=="PM"] = PMDF


,Unnamed: 0,Date,IUCR,Location Description,Arrest,District,Ward,Community Area,Location,VIN,ATTEMPT,RECOVERY
0,41351,06,AUTOMOBILE,STREET,False,6.0,21.0,71.0,"(41.736854324, -87.653368946)",1,0,0
1,385656,07,AUTOMOBILE,STREET,False,8.0,13.0,70.0,"(41.751411416, -87.724549651)",1,0,0
2,289000,16.0,AUTOMOBILE,HOUSE,False,9.0,16.0,61.0,"(41.795034872, -87.674332314)",1,0,0
3,125102,11,AUTOMOBILE,STREET,False,8.0,15.0,66.0,"(41.771627652, -87.705988706)",1,0,0
4,250481,23.0,AUTOMOBILE,STREET,False,12.0,2.0,28.0,"(41.871161623, -87.667801267)",1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
124995,79060,06,"CYCLE, SCOOTER, BIKE",Alcohol,True,14.0,1.0,21.0,"(41.933096182, -87.686514695)",1,0,0
124996,261414,15.0,AUTOMOBILE,STREET,True,5.0,8.0,50.0,"(41.716245774, -87.597018806)",1,0,0
124997,324865,06,AUTOMOBILE,PARKINGZONE,True,19.0,44.0,6.0,"(41.941247675, -87.649308785)",1,0,0
124998,375718,18.0,AUTOMOBILE,STREET,True,18.0,42.0,8.0,"(41.891062111, -87.617043461)",1,0,0


In [22]:
TrainDF.loc[TrainDF['Date'].str[-2:] == "PM", 'Date'] = (    (TrainDF['Date'].str[-11:-9].astype("int64") + 12).astype(str) + TrainDF['Date'].str[-8:])

In [25]:
TrainDF['Date']

0         11/25/2005 06:30:00 AM
1         12/03/2017 07:05:00 AM
2                     1600:00 PM
3         09/16/2008 11:00:00 AM
4                     2330:00 PM
                   ...          
124995    03/27/2006 06:45:00 AM
124996                1500:00 PM
124997    07/18/2012 06:00:00 AM
124998                1800:00 PM
124999    10/05/2002 09:25:52 AM
Name: Date, Length: 125000, dtype: object

In [40]:
# 모두 라벨인코더
Labelencoder = LabelEncoder()

TrainDF['IUCR'] = Labelencoder.fit_transform(TrainDF['IUCR'])
TestDF['IUCR'] = Labelencoder.fit_transform(TestDF['IUCR'])
TrainDF['Location Description'] = Labelencoder.fit_transform(TrainDF['Location Description'])
TestDF['Location Description'] = Labelencoder.fit_transform(TestDF['Location Description'])


# # 자체 Train_test_split
X_train = TrainDF.iloc[:, TrainDF.columns != TrainDF.columns[1]]
y_train = TrainDF[['Arrest']]

X_test = TestDF.iloc[:, TestDF.columns != TestDF.columns[1]]
y_test = TestDF[['Arrest']]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((125000, 11), (125000, 1), (10485, 11), (10485, 1))